# Overfitting and Regularization

In [5]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [8]:
path1 = "../../../data/"
path2 = "../concatenation/data/"

def evaluate_channel(channel, inputs, targets):
    d = channel.d
    r2 = ProbabilityRValue()
    loss = r2(channel, inputs, targets)
    return np.real(loss)


def calculate_channel_fittness(model_name, data_name, num_layers = None):
    model_list = loader(path2 + model_name)
    reps = len(model_list)//3
    
    d = model_list[0].channel.d
    n = int(np.log2(d))

    U1_circuit_list = []
    U2_circuit_list = []
    U3_circuit_list = []

    R1_fid_list = []
    R2_fid_list = []
    R3_fid_list = []

    U1_fid_list = []
    U2_fid_list = []
    U3_fid_list = []

    T1_fid_list = []
    T2_fid_list = []
    T12_fid_list = []
    T3_fid_list = []

    np.random.seed(42)
    random.seed(42)
    
    for i in tqdm(range(reps)):

        circuit1 = pqc_basic(n, num_layers)#.reverse_bits()
        U1 = circuit_to_matrix(circuit1)
        U1_circuit_list.append(U1)

        circuit2 = pqc_basic(n, num_layers)#.reverse_bits()
        U2 = circuit_to_matrix(circuit2)
        U2_circuit_list.append(U2)

        circuit3 = deepcopy(circuit1)
        circuit3 = circuit3.compose(circuit2)
        U3 = circuit_to_matrix(circuit3)
        U3_circuit_list.append(U3)

        #Throw-away computation to advance RNG correctly
        circuit_target_list = [circuit1, circuit2, circuit3]
        belem_concatenate = ExecuteAndCollect()
        belem_concatenate.setup_circuits(circuit_target_list, N_map = 2000-6**n)
    
    for i in range(reps):
        inputs1, targets1, _, _ = loader(path1 + data_name + f"{i}0")
        inputs2, targets2, _, _ = loader(path1 + data_name + f"{i}1")
        inputs3, targets3, _, _ = loader(path1 + data_name + f"{i}2")
        

        model1 = model_list[3*i]
        model2 = model_list[3*i+1]
        model3 = model_list[3*i+2]

        channel1 = model1.channel
        channel2 = model2.channel
        channel3 = model3.channel

        channel12 = channel_to_choi_map([channel1, channel2])
        channel12.spam = channel3.spam

        identity_channel1 = IdentityChannel(d)
        identity_channel1.spam = channel1.spam

        identity_channel2 = IdentityChannel(d)
        identity_channel2.spam = channel2.spam

        identity_channel3 = IdentityChannel(d)
        identity_channel3.spam = channel3.spam

        U_channel1 = ChoiMapStatic(U1_circuit_list[i], mode="unitary")
        U_channel1.spam = channel1.spam

        U_channel2 = ChoiMapStatic(U2_circuit_list[i], mode="unitary")
        U_channel2.spam = channel2.spam

        U_channel3 = ChoiMapStatic(U3_circuit_list[i], mode="unitary")
        U_channel3.spam = channel3.spam

        R1_fid_list.append(evaluate_channel(identity_channel1, inputs1, targets1))
        R2_fid_list.append(evaluate_channel(identity_channel2, inputs2, targets2))
        R3_fid_list.append(evaluate_channel(identity_channel3, inputs3, targets3)) 

        U1_fid_list.append(evaluate_channel(U_channel1, inputs1, targets1))
        U2_fid_list.append(evaluate_channel(U_channel2, inputs2, targets2))
        U3_fid_list.append(evaluate_channel(U_channel3, inputs3, targets3))

        T1_fid_list.append(evaluate_channel(channel1, inputs1, targets1))
        T2_fid_list.append(evaluate_channel(channel2, inputs2, targets2))
        T12_fid_list.append(evaluate_channel(channel12, inputs3, targets3))
        T3_fid_list.append(evaluate_channel(channel3, inputs3, targets3))
        
    
    Id_single_fid_mean = np.mean(R1_fid_list + R2_fid_list)
    Id_single_fid_std= np.std(R1_fid_list + R2_fid_list)
    
    Id_full_fid_mean = np.mean(R3_fid_list)  
    Id_full_fid_std = np.std(R3_fid_list)  

    U_single_fid_mean = np.mean(U1_fid_list + U2_fid_list)
    U_single_fid_std = np.std(U1_fid_list + U2_fid_list)
    
    U_full_fid_mean = np.mean(U3_fid_list)  
    U_full_fid_std = np.std(U3_fid_list)  
    
    T_single_fid_mean = np.mean(T1_fid_list + T2_fid_list)
    T_single_fid_std = np.std(T1_fid_list + T2_fid_list)
    
    T_concat_fid_mean = np.mean(T12_fid_list)
    T_concat_fid_std = np.std(T12_fid_list)
    
    T_full_fid_mean = np.mean(T3_fid_list)
    T_full_fid_std = np.std(T3_fid_list)
    
    mean_list = [Id_single_fid_mean, Id_full_fid_mean, U_single_fid_mean, U_full_fid_mean, T_single_fid_mean, T_full_fid_mean, T_concat_fid_mean]
    std_list = [Id_single_fid_std, Id_full_fid_std, U_single_fid_std, U_full_fid_std, T_single_fid_std, T_full_fid_std, T_concat_fid_std]

    
    return mean_list, std_list

## KL Divergence with Respect to Real Data

In [11]:
mean_list1, std_list1 = calculate_channel_fittness(model_name="belem_concatenate_2layer_fullRank_fitted.model",
                                                   data_name= "belem_concatenate_2layer", 
                                                   num_layers = 2)

mean_list2, std_list2 = calculate_channel_fittness(model_name="belem_concatenate_2layer_halfRank_fitted.model",
                                                   data_name= "belem_concatenate_2layer", 
                                                   num_layers = 2)

mean_list3, std_list3 = calculate_channel_fittness(model_name="belem_concatenate_2layer_quarterRank_fitted.model",
                                                   data_name= "belem_concatenate_2layer", 
                                                   num_layers = 2)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
print(mean_list1[-1])
print(mean_list2[-1])
print(mean_list3[-1])

0.712195671509538
0.7128819955366874
0.7127369104243523


In [17]:
mean_list1, std_list1 = calculate_channel_fittness(model_name="belem_concatenate_4layer_fullRank_fitted.model",
                                                   data_name= "belem_concatenate_4layer", 
                                                   num_layers = 4)

mean_list2, std_list2 = calculate_channel_fittness(model_name="belem_concatenate_4layer_halfRank_fitted.model",
                                                   data_name= "belem_concatenate_4layer", 
                                                   num_layers = 4)

mean_list3, std_list3 = calculate_channel_fittness(model_name="belem_concatenate_4layer_quarterRank_fitted.model",
                                                   data_name= "belem_concatenate_4layer", 
                                                   num_layers = 4)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
print(mean_list1)
print(mean_list2)
print(mean_list3)

[-3.818022424333978, -4.76775661832606, 0.29376551830236286, -0.18336161098261394, 0.934179841522341, 0.934724153784494, 0.6875667436633907]
[-3.8174311358682314, -4.765891860649552, 0.2939924573766093, -0.18228370782826286, 0.9341995517818148, 0.9347604340083816, 0.6846951397046634]
[-3.818081315260947, -4.767929399113197, 0.293824878534763, -0.18344940022298029, 0.933897807697665, 0.9331132018753372, 0.6834505622671028]
